In [28]:
# 📦 Install required packages (if needed)
!pip install pandas folium openrouteservice ipywidgets --quiet
from google.colab import output
output.enable_custom_widget_manager()

# 📚 Imports & Setup
import pandas as pd, folium, openrouteservice, ipywidgets as widgets
from IPython.display import display
from openrouteservice import convert

# 🔑 API key (replace if needed)
api_key = 'eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjliNWQ5OWE0ZWY4MjRmN2VhMmZkZGVmZDY4NThiZTQyIiwiaCI6Im11cm11cjY0In0='
client = openrouteservice.Client(key=api_key)

# 🗂️ Load data
df = pd.read_csv("updated_cities_and_colleges.csv")
cities, colleges = df[df.Type == 'City'], df[df.Type == 'College']

# 💰 Fare logic
fare = lambda d: max(1.2 * d, 10)

# 🎛️ UI dropdowns
dropdowns = {
    'City': widgets.Dropdown(options=cities.Name.tolist(), description='City:', layout=widgets.Layout(width='50%')),
    'College': widgets.Dropdown(options=colleges.Name.tolist(), description='College:', layout=widgets.Layout(width='50%'))
}

# 🧭 Route & Map logic
def on_change(change=None):
    start = df[df.Name == dropdowns['City'].value].iloc[0]
    end = df[df.Name == dropdowns['College'].value].iloc[0]
    coords = ((start.Longitude, start.Latitude), (end.Longitude, end.Latitude))

    try:
        r = client.directions(coords, profile='driving-car', format='geojson')
        geo = r['features'][0]['geometry']['coordinates']
        dist = r['features'][0]['properties']['summary']['distance'] / 1000
    except Exception as e:
        print("⚠️ Route error:", e)
        return

    print(f"\n🚏 {start.Name} ➡ {end.Name}\nDistance: {dist:.2f} km\nFare: ₹{fare(dist):.2f}")

    # Create map
    m = folium.Map(location=[(start.Latitude + end.Latitude)/2, (start.Longitude + end.Longitude)/2], zoom_start=12)

    # Start & End markers
    folium.Marker([start.Latitude, start.Longitude], popup=start.Name, icon=folium.Icon(color='blue')).add_to(m)
    folium.Marker([end.Latitude, end.Longitude], popup=end.Name, icon=folium.Icon(color='green')).add_to(m)

    # Draw route
    latlon_coords = [(y, x) for x, y in geo]
    folium.PolyLine(latlon_coords, color='red', weight=5).add_to(m)

    # Add orange markers with reverse geocoded names
    num_markers = 5
    step = len(latlon_coords) // (num_markers + 1)
    for i in range(1, num_markers + 1):
        point = latlon_coords[i * step]
        try:
            loc_info = client.pelias_reverse(point[::-1])  # (lon, lat)
            name = loc_info['features'][0]['properties'].get('name', 'Unknown')
        except:
            name = f"Point {i}"
        folium.Marker(location=point, icon=folium.Icon(color='orange'), popup=name).add_to(m)

    display(m)

# 🚦 Activate
display(dropdowns['City'], dropdowns['College'])
dropdowns['City'].observe(on_change, names='value')
dropdowns['College'].observe(on_change, names='value')
on_change()


Dropdown(description='City:', layout=Layout(width='50%'), options=('Kakinada', 'Rajahmundry', 'Samalkot', 'Tun…

Dropdown(description='College:', layout=Layout(width='50%'), options=('Pragati Engineering College', 'Aditya E…


🚏 Kakinada ➡ Pragati Engineering College
Distance: 16.82 km
Fare: ₹20.18
